# Embedding Analysis

Research notebook for evaluating embedding quality and comparing LLM classifications against embedding nearest-neighbor baselines.

**Requires:**
- `data/embeddings/voyage_mcp_emb_2026-01-22.npy` — Voyage-4-large MCP embeddings
- `data/embeddings/voyage_dwa_emb.npy` — Voyage-4-large DWA embeddings
- `data/embeddings/mpnet_mcp_emb.npy` — all-mpnet-base-v2 MCP embeddings (optional)
- `data/embeddings/mpnet_dwa_emb.npy` — all-mpnet-base-v2 DWA embeddings (optional)
- `data/mcp/mcp_data_2026-01-22.csv` — cleaned MCP data
- `data/onet/onet_data.csv` — O*NET tasks
- `data/mcp/mcp_classification_teddy.csv` — manual ground-truth classifications
- `data/mcp/gpt-4.1_v5.2_occ_gwa_iwa_dwa_task.csv` — GPT-4.1 v5.2 classifications

The mpnet embeddings are not generated by the main pipeline. To regenerate them:
```python
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-mpnet-base-v2')
embs = np.array(model.encode(texts, show_progress_bar=True), dtype=np.float32)
np.save('data/embeddings/mpnet_mcp_emb.npy', embs)
```

In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

DATA_DIR = Path("../data")

# ---------- Load MCP + O*NET data ----------
# The full Jan 2026 dataset used for the original classification run
mcp_df = pd.read_csv(DATA_DIR / "mcp/raw/mcp_data_2026-01-22.csv")
onet_df = pd.read_csv(DATA_DIR / "onet/onet_data.csv")

mcp_titles = mcp_df["title"].tolist()
dwa_titles_unique = onet_df["dwa_title"].dropna().drop_duplicates().reset_index(drop=True).tolist()

print(f"MCPs loaded:         {len(mcp_df):,}")
print(f"Unique DWA titles:   {len(dwa_titles_unique):,}")

# ---------- Load Voyage embeddings ----------
# voyage_mcp_emb.npy = original full-dataset embeddings (8,957 MCPs x 1024-dim)
voyage_mcp_emb = np.load(DATA_DIR / "embeddings/voyage_mcp_emb.npy")
voyage_dwa_emb = np.load(DATA_DIR / "embeddings/voyage_dwa_emb.npy")
print(f"Voyage MCP emb:  {voyage_mcp_emb.shape}")
print(f"Voyage DWA emb:  {voyage_dwa_emb.shape}")

# ---------- Load mpnet embeddings (optional) ----------
mpnet_mcp_path = DATA_DIR / "embeddings/mpnet_mcp_emb.npy"
mpnet_dwa_path = DATA_DIR / "embeddings/mpnet_dwa_emb.npy"
if mpnet_mcp_path.exists() and mpnet_dwa_path.exists():
    mpnet_mcp_emb = np.load(mpnet_mcp_path)
    mpnet_dwa_emb = np.load(mpnet_dwa_path)
    print(f"mpnet MCP emb:   {mpnet_mcp_emb.shape}")
    print(f"mpnet DWA emb:   {mpnet_dwa_emb.shape}")
    MPNET_AVAILABLE = True
else:
    print("mpnet embeddings not found — skipping mpnet sections.")
    MPNET_AVAILABLE = False

# ---------- L2 normalize ----------
def l2_normalize(X):
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms == 0] = 1
    return X / norms

voyage_mcp_norm = l2_normalize(voyage_mcp_emb)
voyage_dwa_norm = l2_normalize(voyage_dwa_emb)
if MPNET_AVAILABLE:
    mpnet_mcp_norm = l2_normalize(mpnet_mcp_emb)
    mpnet_dwa_norm = l2_normalize(mpnet_dwa_emb)

print("\nSetup complete.")


## Cosine Similarity Distribution Analysis

In [ ]:
# ============================================================
# Cosine Similarity Distribution Analysis
# ============================================================

def print_stats(label, vals):
    print(f"    Mean:        {np.mean(vals):.4f}")
    print(f"    Median:      {np.median(vals):.4f}")
    print(f"    Std Dev:     {np.std(vals):.4f}")
    print(f"    Min:         {np.min(vals):.4f}")
    print(f"    Max:         {np.max(vals):.4f}")
    print(f"    5th pctile:  {np.percentile(vals, 5):.4f}")
    print(f"    25th pctile: {np.percentile(vals, 25):.4f}")
    print(f"    75th pctile: {np.percentile(vals, 75):.4f}")
    print(f"    95th pctile: {np.percentile(vals, 95):.4f}")


def run_distribution_analysis(mcp_emb, dwa_emb, model_name):
    mcp_norm = l2_normalize(mcp_emb)
    dwa_norm = l2_normalize(dwa_emb)

    n_mcp = mcp_norm.shape[0]
    n_dwa = dwa_norm.shape[0]

    print("=" * 70)
    print(f"COSINE SIMILARITY DISTRIBUTION ANALYSIS  --  {model_name}")
    print(f"  {n_mcp:,} MCP servers  |  {n_dwa:,} unique DWA titles")
    print("=" * 70)

    TOP_K = 10
    chunk_size = 500
    top1_sims = []
    topk_means = []
    all_means = []
    rank_sims = [[] for _ in range(TOP_K)]

    for i in range(0, n_mcp, chunk_size):
        chunk = mcp_norm[i : i + chunk_size]
        sims = chunk @ dwa_norm.T

        for row in sims:
            sorted_row = np.sort(row)[::-1]
            top1_sims.append(sorted_row[0])
            topk_means.append(sorted_row[:TOP_K].mean())
            all_means.append(row.mean())
            for k in range(TOP_K):
                rank_sims[k].append(sorted_row[k])

    top1_sims = np.array(top1_sims)
    topk_means = np.array(topk_means)
    all_means = np.array(all_means)

    print()
    print("--- A. MCP Embedding -> DWA Embeddings ---")
    print()
    print("  Top-1 (closest single DWA)")
    print_stats("top1", top1_sims)

    print()
    print(f"  Top-{TOP_K} mean (avg of {TOP_K} closest)")
    print_stats(f"top{TOP_K}", topk_means)

    print()
    print("  All-DWA mean (baseline avg across all DWAs)")
    print_stats("all", all_means)

    print()
    print(f"  Similarity drop-off by rank position (1=closest, {TOP_K}={TOP_K}th closest):")
    print(f"    {'Rank':>8}   {'Mean':>6}   {'Median':>6}      {'Std':>5}")
    for k in range(TOP_K):
        arr = np.array(rank_sims[k])
        print(f"    {k+1:>8}   {arr.mean():.4f}   {np.median(arr):.4f}   {arr.std():.4f}")

    print()
    return mcp_norm, dwa_norm


run_distribution_analysis(voyage_mcp_emb, voyage_dwa_emb, "Voyage-4-large")

if MPNET_AVAILABLE:
    run_distribution_analysis(mpnet_mcp_emb, mpnet_dwa_emb, "all-mpnet-base-v2")

## Example Similarity Pairs (Eyeball Check)

In [ ]:
# ============================================================
# Example pairs at different similarity thresholds
# For each model: show 2 examples each for DWA<->DWA,
# MCP<->MCP, and MCP<->DWA at HIGH and MEDIUM similarity
# ============================================================

def show_example_pairs(mcp_norm, dwa_norm, mcp_titles_list, dwa_titles_list, model_name):
    print("=" * 70)
    print(f"EXAMPLE SIMILARITY PAIRS  --  {model_name}")
    print("=" * 70)

    # --- DWA <-> DWA ---
    print("\n--- DWA <-> DWA ---")
    dwa_sims = dwa_norm @ dwa_norm.T
    np.fill_diagonal(dwa_sims, 0)

    flat_idx = np.argmax(dwa_sims)
    i, j = divmod(flat_idx, dwa_sims.shape[1])
    print(f"\n  HIGH similarity ({dwa_sims[i, j]:.4f}):")
    print(f"    A: {dwa_titles_list[i]}")
    print(f"    B: {dwa_titles_list[j]}")

    upper = dwa_sims[np.triu_indices_from(dwa_sims, k=1)]
    median_sim = np.median(upper)
    diff = np.abs(dwa_sims - median_sim)
    np.fill_diagonal(diff, 999)
    flat_idx = np.argmin(diff)
    i, j = divmod(flat_idx, dwa_sims.shape[1])
    print(f"\n  MEDIUM similarity ({dwa_sims[i, j]:.4f}, median={median_sim:.4f}):")
    print(f"    A: {dwa_titles_list[i]}")
    print(f"    B: {dwa_titles_list[j]}")

    # --- MCP <-> MCP ---
    print("\n--- MCP <-> MCP ---")
    sample_n = min(500, len(mcp_titles_list))
    rng = np.random.default_rng(42)
    sample_idx = rng.choice(len(mcp_titles_list), sample_n, replace=False)
    mcp_sample = mcp_norm[sample_idx]
    mcp_sample_titles = [mcp_titles_list[i] for i in sample_idx]

    mcp_sims = mcp_sample @ mcp_sample.T
    np.fill_diagonal(mcp_sims, 0)

    flat_idx = np.argmax(mcp_sims)
    i, j = divmod(flat_idx, mcp_sims.shape[1])
    print(f"\n  HIGH similarity ({mcp_sims[i, j]:.4f}):")
    print(f"    A: {mcp_sample_titles[i]}")
    print(f"    B: {mcp_sample_titles[j]}")

    upper = mcp_sims[np.triu_indices_from(mcp_sims, k=1)]
    median_sim = np.median(upper)
    diff = np.abs(mcp_sims - median_sim)
    np.fill_diagonal(diff, 999)
    flat_idx = np.argmin(diff)
    i, j = divmod(flat_idx, mcp_sims.shape[1])
    print(f"\n  MEDIUM similarity ({mcp_sims[i, j]:.4f}, median={median_sim:.4f}):")
    print(f"    A: {mcp_sample_titles[i]}")
    print(f"    B: {mcp_sample_titles[j]}")

    # --- MCP <-> DWA ---
    print("\n--- MCP <-> DWA ---")
    cross_sims = mcp_sample @ dwa_norm.T

    flat_idx = np.argmax(cross_sims)
    i, j = divmod(flat_idx, cross_sims.shape[1])
    print(f"\n  HIGH similarity ({cross_sims[i, j]:.4f}):")
    print(f"    MCP: {mcp_sample_titles[i]}")
    print(f"    DWA: {dwa_titles_list[j]}")

    upper = cross_sims.flatten()
    median_sim = np.median(upper)
    diff = np.abs(cross_sims - median_sim)
    flat_idx = np.argmin(diff)
    i, j = divmod(flat_idx, cross_sims.shape[1])
    print(f"\n  MEDIUM similarity ({cross_sims[i, j]:.4f}, median={median_sim:.4f}):")
    print(f"    MCP: {mcp_sample_titles[i]}")
    print(f"    DWA: {dwa_titles_list[j]}")

    print()


show_example_pairs(voyage_mcp_norm, voyage_dwa_norm, mcp_titles, dwa_titles_unique, "Voyage-4-large")

if MPNET_AVAILABLE:
    show_example_pairs(mpnet_mcp_norm, mpnet_dwa_norm, mcp_titles, dwa_titles_unique, "all-mpnet-base-v2")

## Classification Comparison: Manual + GPT-4.1 v5.2 vs Embedding Nearest Neighbors

For each (MCP, selected DWA) pair in the ground-truth classifications, this section computes the cosine similarity rank of that DWA relative to all 2,083 unique DWAs. This validates how well the embedding retrieval step captures the correct DWAs.

In [ ]:
# ============================================================
# Compare Teddy & GPT-4.1 DWA selections against embedding
# nearest neighbors.
# ============================================================

teddy_class = pd.read_csv(DATA_DIR / "mcp/mcp_classification_teddy.csv")
gpt_class = pd.read_csv(DATA_DIR / "mcp/gpt-4.1_v5.2_occ_gwa_iwa_dwa_task.csv")

print(f"Teddy classifications loaded: {len(teddy_class)} MCPs")
print(f"GPT-4.1 classifications loaded: {len(gpt_class)} MCPs")

# --- Build lookup tables ---
def normalize_dwa_text(text):
    text = text.strip().lower()
    text = text.rstrip(".")
    text = re.sub(r"[,;]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

dwa_lookup = {normalize_dwa_text(d): i for i, d in enumerate(dwa_titles_unique)}
mcp_title_to_idx = {str(t).strip(): i for i, t in enumerate(mcp_titles)}
mcp_url_to_idx = {str(u).strip(): i for i, u in enumerate(mcp_df["url"].values)}

print(f"DWA lookup:  {len(dwa_lookup):,} entries")
print(f"MCP lookup:  {len(mcp_title_to_idx):,} entries")


def compute_dwa_ranks(class_df, source_name, mcp_norm, dwa_norm, dwa_col="dwa"):
    results = []
    n_matched_mcps = 0
    n_unmatched_mcps = 0
    n_matched_dwas = 0
    n_unmatched_dwas = 0
    unmatched_examples = []

    valid = class_df[
        class_df[dwa_col].notna()
        & (class_df[dwa_col].astype(str).str.strip() != "")
        & (class_df["occ_relevant"].astype(str).str.strip().str.lower() == "yes")
    ].copy()

    for _, row in valid.iterrows():
        title = str(row["title"]).strip()
        url = str(row.get("url", "")).strip()

        mcp_idx = mcp_title_to_idx.get(title)
        if mcp_idx is None:
            mcp_idx = mcp_url_to_idx.get(url)
        if mcp_idx is None:
            n_unmatched_mcps += 1
            continue

        mcp_vec = mcp_norm[mcp_idx : mcp_idx + 1]
        n_matched_mcps += 1

        all_sims = (mcp_vec @ dwa_norm.T).flatten()
        sorted_indices = np.argsort(-all_sims)

        rank_lookup = np.empty(len(all_sims), dtype=int)
        rank_lookup[sorted_indices] = np.arange(1, len(all_sims) + 1)

        top1_sim = float(all_sims[sorted_indices[0]])
        top1_dwa_text = dwa_titles_unique[sorted_indices[0]]

        dwa_strings = [t.strip() for t in str(row[dwa_col]).split(";") if t.strip()]

        for dwa_str in dwa_strings:
            norm_key = normalize_dwa_text(dwa_str)
            if norm_key in dwa_lookup:
                dwa_idx = dwa_lookup[norm_key]
                sim = float(all_sims[dwa_idx])
                rank = int(rank_lookup[dwa_idx])
                n_matched_dwas += 1
                results.append({
                    "source": source_name,
                    "mcp_title": title,
                    "selected_dwa": dwa_str,
                    "cosine_similarity": round(sim, 4),
                    "rank": rank,
                    "total_dwas": len(dwa_titles_unique),
                    "percentile": round((1 - rank / len(dwa_titles_unique)) * 100, 2),
                    "top1_similarity": round(top1_sim, 4),
                    "top1_dwa": top1_dwa_text,
                    "sim_gap_from_top1": round(top1_sim - sim, 4),
                })
            else:
                n_unmatched_dwas += 1
                unmatched_examples.append((title, dwa_str))

    print(f"\n  {source_name}")
    print(f"  MCPs matched: {n_matched_mcps}  |  not found: {n_unmatched_mcps}")
    print(f"  DWAs matched: {n_matched_dwas}  |  not matched: {n_unmatched_dwas}")
    if unmatched_examples:
        print("  Sample unmatched DWAs:")
        for title, dwa in unmatched_examples[:5]:
            print(f"    [{title}] {dwa[:80]}")

    return pd.DataFrame(results)


def print_rank_summary(label, df):
    if df.empty:
        print(f"{label}: No matched DWAs to analyze.")
        return
    print()
    print("=" * 60)
    print(f"  {label} -- Summary Statistics")
    print("=" * 60)
    print(f"  Total (MCP, DWA) pairs: {len(df):,}")
    print(f"  Unique MCPs:            {df['mcp_title'].nunique()}")
    print()
    print("  Cosine Similarity of Selected DWAs:")
    print(f"    Mean:    {df['cosine_similarity'].mean():.4f}")
    print(f"    Median:  {df['cosine_similarity'].median():.4f}")
    print()
    print(f"  Rank among {df['total_dwas'].iloc[0]:,} unique DWAs:")
    print(f"    Mean rank:   {df['rank'].mean():.0f}")
    print(f"    Median rank: {df['rank'].median():.0f}")
    print(f"    % in top 10:   {(df['rank'] <= 10).mean()*100:.1f}%")
    print(f"    % in top 50:   {(df['rank'] <= 50).mean()*100:.1f}%")
    print(f"    % in top 80:   {(df['rank'] <= 80).mean()*100:.1f}%")
    print(f"    % in top 100:  {(df['rank'] <= 100).mean()*100:.1f}%")


models_to_run = [('Voyage-4-large', voyage_mcp_norm, voyage_dwa_norm)]
if MPNET_AVAILABLE:
    models_to_run.append(('all-mpnet-base-v2', mpnet_mcp_norm, mpnet_dwa_norm))

for model_label, mcp_n, dwa_n in models_to_run:
    print('\n' + '#' * 70)
    print(f'#  {model_label}')
    print('#' * 70)

    teddy_ranks = compute_dwa_ranks(teddy_class, f'Teddy (manual) [{model_label}]', mcp_n, dwa_n)
    gpt_ranks = compute_dwa_ranks(gpt_class, f'GPT-4.1 (v5.2) [{model_label}]', mcp_n, dwa_n)

    print_rank_summary(f'Teddy (manual) [{model_label}]', teddy_ranks)
    print_rank_summary(f'GPT-4.1 (v5.2) [{model_label}]', gpt_ranks)
